In [1]:
#coding=utf-8
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn import preprocessing  
min_max_scaler = preprocessing.MinMaxScaler()  
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import train_test_split

llb


In [2]:
data = pd.read_csv("../inputnew/contest_ext_crd_is_ovdsummary.csv")
print(data.shape)
print(data.columns)
data.head()

(76212, 6)
Index(['REPORT_ID', 'TYPE_DW', 'COUNT_DW', 'MONTHS', 'HIGHEST_OA_PER_MON',
       'MAX_DURATION'],
      dtype='object')


,REPORT_ID,TYPE_DW,COUNT_DW,MONTHS,HIGHEST_OA_PER_MON,MAX_DURATION
0,9410,1,0,0,0.0,0
1,9410,2,1,1,0.0,1
2,9410,0,0,0,0.0,0
3,24820,1,0,0,0.0,0
4,24820,2,3,6,440.0,1


In [3]:
data.isnull().sum()

REPORT_ID             0
TYPE_DW               0
COUNT_DW              0
MONTHS                0
HIGHEST_OA_PER_MON    0
MAX_DURATION          0
dtype: int64

In [6]:
def groupDeal(group):
    # print(group)
    TYPE_DW_more = group.TYPE_DW.value_counts().iloc[0]
    TYPE_DW_less = group.TYPE_DW.value_counts().iloc[-1]
    TYPE_DW_num = group.TYPE_DW.shape[0]

    COUNT_DW_max = group.COUNT_DW.max()
    COUNT_DW_min = group.COUNT_DW.min()
    COUNT_DW_mean = round(group.COUNT_DW.mean())
    COUNT_DW_zongshu = group.COUNT_DW.value_counts().iloc[0]
    COUNT_DW_num = group.COUNT_DW.shape[0]

    MONTHS_max = group.MONTHS.max()
    MONTHS_min = group.MONTHS.min()
    MONTHS_mean = round(group.MONTHS.mean())
    MONTHS_zongshu = group.MONTHS.value_counts().iloc[0]
    MONTHS_num = group.MONTHS.shape[0]

    MAX_DURATION_max = group.MAX_DURATION.max()
    MAX_DURATION_min = group.MAX_DURATION.min()
    MAX_DURATION_mean = round(group.MAX_DURATION.mean())
    MAX_DURATION_zongshu = group.MAX_DURATION.value_counts().iloc[0]
    MAX_DURATION_num = group.MAX_DURATION.shape[0]

    HIGHEST_OA_PER_MON_max = group.HIGHEST_OA_PER_MON.max()
    HIGHEST_OA_PER_MON_min = group.HIGHEST_OA_PER_MON.min()
    HIGHEST_OA_PER_MON_mean = group.HIGHEST_OA_PER_MON.mean()
    HIGHEST_OA_PER_MON_zongshu = group.HIGHEST_OA_PER_MON.value_counts().iloc[0]
    HIGHEST_OA_PER_MON_num = group.HIGHEST_OA_PER_MON.shape[0]

    indexs = {
        "REPORT_ID":int(group.REPORT_ID.iloc[0]),
        "TYPE_DW_num": TYPE_DW_num,

        "COUNT_DW_max": COUNT_DW_max,
        "COUNT_DW_min": COUNT_DW_min,
        "COUNT_DW_mean": COUNT_DW_mean,
        "COUNT_DW_zongshu": COUNT_DW_zongshu,
        "COUNT_DW_num": COUNT_DW_num,

        "MONTHS_max": MONTHS_max,
        "MONTHS_min": MONTHS_min,
        "MONTHS_mean": MONTHS_mean,
        "MONTHS_zongshu": MONTHS_zongshu,
        "MONTHS_num": MONTHS_num,

        "HIGHEST_OA_PER_MON_max": HIGHEST_OA_PER_MON_max,
        "HIGHEST_OA_PER_MON_min": HIGHEST_OA_PER_MON_min,
        "HIGHEST_OA_PER_MON_mean": HIGHEST_OA_PER_MON_mean,
        "HIGHEST_OA_PER_MON_zongshu": HIGHEST_OA_PER_MON_zongshu,
        "HIGHEST_OA_PER_MON_num": HIGHEST_OA_PER_MON_num,

        "MAX_DURATION_max": MAX_DURATION_max,
        "MAX_DURATION_min": MAX_DURATION_min,
        "MAX_DURATION_mean": MAX_DURATION_mean,
        "MAX_DURATION_zongshu": MAX_DURATION_zongshu,
        "MAX_DURATION_num": MAX_DURATION_num,
        
         "TYPE_DW_more": TYPE_DW_more,
        "TYPE_DW_less": TYPE_DW_less
    }
    return pd.Series(data=[indexs[c] for c in indexs], index=[c for c in indexs])

In [8]:
# tmp1 = data.iloc[0:30,:].groupby(by=['REPORT_ID']).apply(groupDeal)
tmp1 = data.groupby(by=['REPORT_ID']).apply(groupDeal)
# tmp1
tmp1.to_csv("../add/willnorm/contest_ext_crd_is_ovdsummaryRaw.csv",index = False)

In [9]:
data = pd.read_csv("../add/willnorm/contest_ext_crd_is_ovdsummaryRaw.csv")
print(data.shape)
print(data.columns)
data.head()

(25404, 24)
Index(['REPORT_ID', 'TYPE_DW_num', 'COUNT_DW_max', 'COUNT_DW_min',
       'COUNT_DW_mean', 'COUNT_DW_zongshu', 'COUNT_DW_num', 'MONTHS_max',
       'MONTHS_min', 'MONTHS_mean', 'MONTHS_zongshu', 'MONTHS_num',
       'HIGHEST_OA_PER_MON_max', 'HIGHEST_OA_PER_MON_min',
       'HIGHEST_OA_PER_MON_mean', 'HIGHEST_OA_PER_MON_zongshu',
       'HIGHEST_OA_PER_MON_num', 'MAX_DURATION_max', 'MAX_DURATION_min',
       'MAX_DURATION_mean', 'MAX_DURATION_zongshu', 'MAX_DURATION_num',
       'TYPE_DW_more', 'TYPE_DW_less'],
      dtype='object')


,REPORT_ID,TYPE_DW_num,COUNT_DW_max,COUNT_DW_min,COUNT_DW_mean,COUNT_DW_zongshu,COUNT_DW_num,MONTHS_max,MONTHS_min,MONTHS_mean,...,HIGHEST_OA_PER_MON_mean,HIGHEST_OA_PER_MON_zongshu,HIGHEST_OA_PER_MON_num,MAX_DURATION_max,MAX_DURATION_min,MAX_DURATION_mean,MAX_DURATION_zongshu,MAX_DURATION_num,TYPE_DW_more,TYPE_DW_less
0,9410.0,3.0,1.0,0.0,0.0,2.0,3.0,1.0,0.0,0.0,...,0.000000,3.0,3.0,1.0,0.0,0.0,2.0,3.0,1.0,1.0
1,22999.0,3.0,1.0,0.0,0.0,2.0,3.0,1.0,0.0,0.0,...,0.000000,3.0,3.0,1.0,0.0,0.0,2.0,3.0,1.0,1.0
2,24820.0,3.0,3.0,0.0,1.0,2.0,3.0,6.0,0.0,2.0,...,146.666667,2.0,3.0,1.0,0.0,0.0,2.0,3.0,1.0,1.0
3,41549.0,3.0,5.0,0.0,2.0,2.0,3.0,3.0,0.0,1.0,...,1155.666667,2.0,3.0,1.0,0.0,0.0,2.0,3.0,1.0,1.0
4,48614.0,3.0,3.0,0.0,1.0,1.0,3.0,3.0,0.0,2.0,...,36.000000,2.0,3.0,1.0,0.0,1.0,2.0,3.0,1.0,1.0


In [10]:
# 6 个类别 *2
label =['TYPE_DW_more', 'TYPE_DW_less']
#
cate = []
c = 0
size = 0
for i in range(len(label)):
    tmp = []
    tmp = pd.get_dummies(data[label[i]],prefix = label[i])
    size += tmp.shape[1]
    if c == 0:
        cate = tmp
        c = 3
        continue
    cate = pd.concat([cate,tmp],axis = 1)
cate.head()

,TYPE_DW_more_1.0,TYPE_DW_less_1.0
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1


In [11]:
label = data.iloc[:,1:-2].columns
# print(label)
number = data.iloc[:,1:-2]
kk = min_max_scaler.fit_transform(number)  
numbernew = pd.DataFrame(kk,columns = label)
numbernew.head()
fulldata = pd.concat([data.REPORT_ID,numbernew],axis = 1)
fulldata.head()

,REPORT_ID,TYPE_DW_num,COUNT_DW_max,COUNT_DW_min,COUNT_DW_mean,COUNT_DW_zongshu,COUNT_DW_num,MONTHS_max,MONTHS_min,MONTHS_mean,...,HIGHEST_OA_PER_MON_max,HIGHEST_OA_PER_MON_min,HIGHEST_OA_PER_MON_mean,HIGHEST_OA_PER_MON_zongshu,HIGHEST_OA_PER_MON_num,MAX_DURATION_max,MAX_DURATION_min,MAX_DURATION_mean,MAX_DURATION_zongshu,MAX_DURATION_num
0,9410.0,0.0,0.000000,0.0,0.000000,0.5,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.000000,0.5,0.0
1,22999.0,0.0,0.000000,0.0,0.000000,0.5,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.000000,0.5,0.0
2,24820.0,0.0,0.038462,0.0,0.055556,0.5,0.0,0.084746,0.0,0.068966,...,0.000110,0.0,0.000108,0.5,0.0,0.0,0.0,0.000000,0.5,0.0
3,41549.0,0.0,0.076923,0.0,0.111111,0.5,0.0,0.033898,0.0,0.034483,...,0.000865,0.0,0.000852,0.5,0.0,0.0,0.0,0.000000,0.5,0.0
4,48614.0,0.0,0.038462,0.0,0.055556,0.0,0.0,0.033898,0.0,0.068966,...,0.000027,0.0,0.000027,0.5,0.0,0.0,0.0,0.142857,0.5,0.0


In [12]:
#删掉一些维度
train = pd.read_csv("../input/contest_basic_train.tsv",sep='\t')
mm = pd.concat([train.REPORT_ID,train.Y],axis = 1)
ceshidata = fulldata.merge(mm,how = 'inner',left_on = 'REPORT_ID',right_on = 'REPORT_ID') 
print(ceshidata.head())
print(ceshidata.shape)

  REPORT_ID  TYPE_DW_num  COUNT_DW_max  COUNT_DW_min  COUNT_DW_mean  \
0      9410          0.0      0.000000           0.0       0.000000   
1     22999          0.0      0.000000           0.0       0.000000   
2     24820          0.0      0.038462           0.0       0.055556   
3     41549          0.0      0.076923           0.0       0.111111   
4     48614          0.0      0.038462           0.0       0.055556   

   COUNT_DW_zongshu  COUNT_DW_num  MONTHS_max  MONTHS_min  MONTHS_mean ...  \
0               0.5           0.0    0.000000         0.0     0.000000 ...   
1               0.5           0.0    0.000000         0.0     0.000000 ...   
2               0.5           0.0    0.084746         0.0     0.068966 ...   
3               0.5           0.0    0.033898         0.0     0.034483 ...   
4               0.0           0.0    0.033898         0.0     0.068966 ...   

   HIGHEST_OA_PER_MON_min  HIGHEST_OA_PER_MON_mean  \
0                     0.0                 0.000000

In [13]:
ceshidata.Y.value_counts()

0    18384
1     1219
Name: Y, dtype: int64

In [8]:
# from imblearn.over_sampling import SMOTE, ADASYN
# X_resampled, y_resampled = SMOTE(kind='borderline1').fit_sample(ceshidata.iloc[:,1:-1], ceshidata.iloc[:,-1])

In [14]:

# print(X_resampled[0:10,:])
# print(X_resampled.shape)
# print(y_resampled[0:10])
# print(y_resampled.shape)
# pd.Series(y_resampled).value_counts()
# x_train,x_valid,y_train, y_valid= train_test_split(X_resampled,y_resampled,test_size = 0.3)
x_train,x_valid,y_train, y_valid= train_test_split(ceshidata.iloc[:,1:-1],ceshidata.iloc[:,-1],test_size = 0.3)
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(13722, 21)
(13722,)
(5881, 21)
(5881,)


In [11]:
pd.Series(y_train).value_counts()

0    19626
1     1321
Name: Y, dtype: int64

In [15]:
# x_train = ceshidata.iloc[0:18000,1:-1]
# y_train = ceshidata.iloc[0:18000,-1]
# x_valid = ceshidata.iloc[18000:,1:-1]
# y_valid = ceshidata.iloc[18000:,-1]

skf = StratifiedKFold(n_splits=5)
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=3,
        scale_pos_weight=1,
        StratifiedKFold = True,
        seed=27)
xgb_param = xgb1.get_xgb_params()
#构建稀疏矩阵，运行更快
xgtrain = xgb.DMatrix(x_train, label = y_train)
# xgtest =xgb.DMatrix(dtest[predictors].values)
mm = xgb.cv(xgb_param, xgtrain, num_boost_round=1000, nfold=3,
    metrics='auc', early_stopping_rounds=50)
mm

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.561338,0.013063,0.579767,0.012729
1,0.568072,0.020313,0.590057,0.011613
2,0.574181,0.023718,0.601916,0.009827
3,0.575062,0.022874,0.603677,0.012429
4,0.577118,0.024677,0.624632,0.008710
5,0.577796,0.024943,0.628547,0.009037
6,0.581842,0.024467,0.632118,0.011499
7,0.584352,0.024439,0.637021,0.011468
8,0.583571,0.021418,0.645213,0.012278
9,0.585255,0.020974,0.653931,0.008947


In [16]:

print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(13722, 21)
(13722,)
(5881, 21)
(5881,)


In [17]:
feature = 0
for i in range(5):
    xgb2 = XGBClassifier(
            learning_rate =0.1,
            n_estimators=mm.shape[0],
            max_depth=5,
            min_child_weight=1,
            gamma=0,
            subsample=0.8,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,   #15:1
            StratifiedKFold = True,
            seed=27)
    xgb2.fit(x_train,y_train)
    feature += xgb2.feature_importances_
    print(i)
#     pred = xgb2.predict_proba(x_valid)[:,1]
#     print(pred)
#     auc = roc_auc_score(y_valid, pred)  
#     auc
feature = feature/5

0
1
2
3
4


In [18]:
feature

array([0.        , 0.10162161, 0.00216216, 0.01837838, 0.01621622,
       0.        , 0.14054054, 0.01081081, 0.08756757, 0.00648649,
       0.        , 0.19351351, 0.00432432, 0.23783784, 0.01081081,
       0.        , 0.10702703, 0.00108108, 0.04648649, 0.01513514,
       0.        ], dtype=float32)

In [19]:
print(x_train.columns)
print(feature[feature>0.0000008])
x_train.columns[feature>0.0000008]

Index(['TYPE_DW_num', 'COUNT_DW_max', 'COUNT_DW_min', 'COUNT_DW_mean',
       'COUNT_DW_zongshu', 'COUNT_DW_num', 'MONTHS_max', 'MONTHS_min',
       'MONTHS_mean', 'MONTHS_zongshu', 'MONTHS_num', 'HIGHEST_OA_PER_MON_max',
       'HIGHEST_OA_PER_MON_min', 'HIGHEST_OA_PER_MON_mean',
       'HIGHEST_OA_PER_MON_zongshu', 'HIGHEST_OA_PER_MON_num',
       'MAX_DURATION_max', 'MAX_DURATION_min', 'MAX_DURATION_mean',
       'MAX_DURATION_zongshu', 'MAX_DURATION_num'],
      dtype='object')
[0.10162161 0.00216216 0.01837838 0.01621622 0.14054054 0.01081081
 0.08756757 0.00648649 0.19351351 0.00432432 0.23783784 0.01081081
 0.10702703 0.00108108 0.04648649 0.01513514]


Index(['COUNT_DW_max', 'COUNT_DW_min', 'COUNT_DW_mean', 'COUNT_DW_zongshu',
       'MONTHS_max', 'MONTHS_min', 'MONTHS_mean', 'MONTHS_zongshu',
       'HIGHEST_OA_PER_MON_max', 'HIGHEST_OA_PER_MON_min',
       'HIGHEST_OA_PER_MON_mean', 'HIGHEST_OA_PER_MON_zongshu',
       'MAX_DURATION_max', 'MAX_DURATION_min', 'MAX_DURATION_mean',
       'MAX_DURATION_zongshu'],
      dtype='object')

In [20]:
fulldata.head()
fulldataok = []
fulldataok = fulldata[x_train.columns[feature>0.0000008]]  #一定要小心是谁的ID
fulldataok = pd.concat([fulldata.REPORT_ID,fulldataok],axis = 1)
fulldataok.head()

,REPORT_ID,COUNT_DW_max,COUNT_DW_min,COUNT_DW_mean,COUNT_DW_zongshu,MONTHS_max,MONTHS_min,MONTHS_mean,MONTHS_zongshu,HIGHEST_OA_PER_MON_max,HIGHEST_OA_PER_MON_min,HIGHEST_OA_PER_MON_mean,HIGHEST_OA_PER_MON_zongshu,MAX_DURATION_max,MAX_DURATION_min,MAX_DURATION_mean,MAX_DURATION_zongshu
0,9410.0,0.000000,0.0,0.000000,0.5,0.000000,0.0,0.000000,0.5,0.000000,0.0,0.000000,1.0,0.0,0.0,0.000000,0.5
1,22999.0,0.000000,0.0,0.000000,0.5,0.000000,0.0,0.000000,0.5,0.000000,0.0,0.000000,1.0,0.0,0.0,0.000000,0.5
2,24820.0,0.038462,0.0,0.055556,0.5,0.084746,0.0,0.068966,0.5,0.000110,0.0,0.000108,0.5,0.0,0.0,0.000000,0.5
3,41549.0,0.076923,0.0,0.111111,0.5,0.033898,0.0,0.034483,0.5,0.000865,0.0,0.000852,0.5,0.0,0.0,0.000000,0.5
4,48614.0,0.038462,0.0,0.055556,0.0,0.033898,0.0,0.068966,0.0,0.000027,0.0,0.000027,0.5,0.0,0.0,0.142857,0.5


In [21]:
fulldataok.to_csv("../add/contest_ext_crd_is_ovdsummaryok.csv",index = False)

In [22]:
fulldataok.shape

(25404, 17)